In [1]:

import sympy as sp
from IPython.display import display
import random

import symqups
from symqups import s as CahillGlauberS
from symqups import *
from symqups._internal.multiprocessing import *
from symqups.objects.scalars import *
from symqups.objects.scalars import t
from symqups.objects.operators import *
from symqups.manipulations import *
from symqups.ordering import *
from symqups.quantization import *
from symqups.bopp import *
from symqups.cg import *
from symqups.eom import *
from symqups.objects import *
from symqups._internal.cache import *
from symqups.star import *
from symqups._internal.basic_routines import operation_routine
from symqups._internal.operator_handling import (
    separate_operator,
    separate_term_by_polynomiality,
    separate_term_oper_by_sub,
    collect_alpha_type_oper_from_monomial_by_sub,
    get_oper_sub
)
from symqups._internal.warnings import suppress_warning_repeats
from symqups._internal.grouping import CannotBoppShift, NotAnOperator, PhaseSpaceObject
from symqups.utils import *
from typing import Sequence

# a = [alpha(i) for i in range(10)]
# aop = [annihilateOp(i) for i in range(10)]
# ad = [alphaD(i) for i in range(10)]
# adop = [createOp(i) for i in range(10)]
# x = sp.Symbol("x")
# qq = [q(i) for i in range(10)]
# pp = [p(i) for i in range(10)]
# CahillGlauberS.val = sp.Rational(1/2)
# # symqups.hbar.val = sp.Number(1)
# CahillGlauberS
a = alpha()
aop = annihilateOp()
ad = alphaD()
adop = createOp()
pi = pi.val
zeta = zeta.val
CahillGlauberS.val = sp.Symbol("s")
CahillGlauberS.val = 0
s = CahillGlauberS.val
hbar = hbar.val
x = sp.Symbol("x")

##

MP_CONFIG["enable"] = True

f = sp.Function("f")(a,ad)
g = sp.Function("g")(a,ad)
F = sp.Function("F")(aop, adop)
G = sp.Function("G")(aop, adop)

In [8]:
from symqups.simple import *
import sympy as sp
import symqups as sq
import time
import matplotlib as mpl
from IPython.display import display

MP_CONFIG['enable']=False

import itertools
for combos in itertools.product((1,2), (3,4), (5,6)):
    print(combos)

(1, 3, 5)
(1, 3, 6)
(1, 4, 5)
(1, 4, 6)
(2, 3, 5)
(2, 3, 6)
(2, 4, 5)
(2, 4, 6)


In [9]:
x = rho
op = sq.HilbertSpaceBoppSuperoperator(aOp)
for _ in range(18):
    x = op.act(x)